### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import pickle

from rules_searcher import *

from sklearn.model_selection import train_test_split

### Выгрузка данных

In [2]:
#Выгрузил данные(необязательно пиклом, можно и просто экселем)
df1 = pd.read_pickle('data/all_data_picke.pkl')

In [3]:
df1 = df1[(df1['of_on']=='online')&(df1['new_rec']=='rec')]

In [4]:
df1['date_requested'] = df1.calculation_date
df1.drop(columns=['calculation_date'], axis=1, inplace=True)

### Преподготовка данных

In [5]:
# Filling missing values (np.nan, '_MISSING_')
cat_vars = df1.select_dtypes(include=[object]).columns
df1 = filling(df1)

min_num = 50
# Можно задать список признаков, который не будет учавствовать в предобработке
ignore_vars = ['date_requested', 'credit_id', 'target']
df1, else_df = replace_not_frequent_2(df1, cat_vars, num_min=min_num, value_to_replace = "_ELSE_", ignore_vars=ignore_vars)

# Drop columns with only 1 value
df_res = drop_single_value_column(df1, except_cols=ignore_vars)

c:\Users\m.skrebtsov\Desktop\Projects\Rules_Framework\rules_searcher.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else_df = else_df.append(pd.DataFrame([[i, q]], columns=['var', 'list']))
c:\Users\m.skrebtsov\Desktop\Projects\Rules_Framework\rules_searcher.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else_df = else_df.append(pd.DataFrame([[i, q]], columns=['var', 'list']))
c:\Users\m.skrebtsov\Desktop\Projects\Rules_Framework\rules_searcher.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else_df = else_df.append(pd.DataFrame([[i, q]], columns=['var', 'list']))
c:\Users\m.skrebtsov\Desktop\Projects\Rules_Framework\rules_searcher.py:33: FutureWarning: The frame.append method is deprecated and will be remo

In [6]:
df_res.head()

,credit_id,borrower_id,product_group_new,target,days_from_first_log_in,days_from_last_log_in,cnt_log_in_last_30_days,cnt_log_in_last_60_days,cnt_log_in_last_120_days,cnt_log_in_last_360_days,...,industry_current,io_device_type,io_device_browser_type,settlement_type,reg_date,employment_y,ip,qty_appl,region,date_requested
7,4521733,356721,MICRO,0,1489,0,3,3,3,3,...,industry.other,_MISSING_,_MISSING_,settlement.type.selo,_MISSING_,employment.selfEmployed,1.0,2,KARAGANDINSKAIA,2023-04-01
29,4521847,463270,TOP_UP_SME,1,1370,0,18,43,70,161,...,industry.sales,_MISSING_,_MISSING_,settlement.type.city,_MISSING_,employment.formyself,1.0,3,VOSTOCHNOKAZ,2023-04-01
30,4521849,1172835,MICRO_SELF_EMPLOYED,0,371,0,7,32,40,103,...,industry.other,_MISSING_,_MISSING_,settlement.type.area,_MISSING_,employment.selfEmployed,NaN,3,ALMATI,2023-04-01
31,4521856,297804,MICRO_SELF_EMPLOYED,1,1559,0,2,6,10,26,...,industry.govservice,_MISSING_,_MISSING_,settlement.type.selo,_MISSING_,employment.selfEmployed,NaN,10,KIZILORDINSKAIA,2023-04-01
32,4521897,975362,SME,0,612,0,2,4,4,38,...,industry.govservice,_MISSING_,_MISSING_,settlement.type.city,_ELSE_,employment.publicSectorEmployee,1.0,22,ZHETYSU,2023-04-01


### Разбиение на тренировочную, тестовую и out-выборку

In [7]:
#Можно добавить OUT-выборку (По желанию)

cnt = int(df_res.shape[0]*0.75)
df_out = df_res.sort_values(by='date_requested')[cnt:]
y_out = df_out.target
df = df_res.sort_values(by='date_requested')[:cnt]

In [8]:
df_train, df_test, y_train, y_test = train_test_split(df, df.target, test_size=0.2, stratify=df.target, random_state=42)

In [9]:
print('BR in train sample: ', round(y_train.mean(), 3), ', train_sample size: ', df_train.shape[0])
print('BR in test sample: ', round(y_test.mean(), 3), ', test_sample size: ', df_test.shape[0])
print('BR in out sample:', round(y_out.mean(), 3), ', out_sample size: ', df_out.shape[0])

BR in train sample:  0.444 , train_sample size:  13758
BR in test sample:  0.444 , test_sample size:  3440
BR in out sample: 0.462 , out_sample size:  5733


In [10]:
df_train.head()

,credit_id,borrower_id,product_group_new,target,days_from_first_log_in,days_from_last_log_in,cnt_log_in_last_30_days,cnt_log_in_last_60_days,cnt_log_in_last_120_days,cnt_log_in_last_360_days,...,industry_current,io_device_type,io_device_browser_type,settlement_type,reg_date,employment_y,ip,qty_appl,region,date_requested
49707,4803567,1098715,TOP_UP_SME,1,549,0,8,8,13,41,...,industry.sales,_MISSING_,_MISSING_,settlement.type.selo,_MISSING_,employment.selfEmployed,NaN,2,ALMATINSKAIA,2023-07-20
87839,5094890,327458,TOP_UP_SME,1,1737,0,12,26,45,170,...,industry.other,_MISSING_,_MISSING_,settlement.type.city,_MISSING_,employment.formyself,1.0,5,KIZILORDINSKAIA,2023-11-02
78968,5020447,729237,SME,0,1171,0,22,53,121,338,...,industry.sales,_MISSING_,_MISSING_,settlement.type.city,_MISSING_,employment.rate,NaN,10,AKTUBINSKAIA,2023-10-06
65664,4896804,1495966,SME,1,91,0,1,1,26,26,...,industry.other,_MISSING_,_MISSING_,settlement.type.city,_ELSE_,employment.formyself,1.0,1,KARAGANDINSKAIA,2023-08-24
17970,4618241,1090155,SME,0,499,0,1,2,5,43,...,industry.sales,_MISSING_,_MISSING_,_MISSING_,_MISSING_,employment.workingOnSe,1.0,3,SHYMKENT,2023-05-11


In [11]:
# Записать сюда переменные, которые не должны принимать участия в построении рулов
exclude_columns = ['target', 'credit_id', 'date_requested', 'product_group_new', 'borrower_id']

In [12]:
all_feats = df_train.drop(columns=exclude_columns, axis=1).columns

In [13]:
cat_feat = df_train.drop(columns=exclude_columns, axis=1).select_dtypes(include=[object]).columns

In [14]:
cat_encoding = fit_category_encoding(df_train, cat_feat, encoding_type='target_encoding')

X_train = transform_category_encoding(df_train, cat_feat, cat_encoding)
X_test = transform_category_encoding(df_test, cat_feat, cat_encoding)
X_out = transform_category_encoding(df_out, cat_feat, cat_encoding)

In [15]:
num_feat = df_train.drop(columns=exclude_columns, axis=1).select_dtypes(exclude=object).columns
nan_custom = {
    # 'cnt_IL_all': 5,
}
# Указываем способ заполнения пропусков для отсальных признаков.
# 'median' - заполнение медианным значением
# 'max_freq' - заполнение значением с наибольшей частотой
# 'constant' - заполнение константой, при этом надо указать эту константу
#       в параметре fill_value
# 'auto_iv' - заполнение афтоматически, исходя из подходящих бинов в WOE таблице,
#       при этом надо указать параметр iv_df
nan_imputer = 'median'

nan_encoding = fit_nan_encoding(df_train, num_feat, nan_imputer, nan_custom, iv_df=None)

X_train = transform_nan_encoding(X_train, num_feat, nan_encoding)
X_test = transform_nan_encoding(X_test, num_feat, nan_encoding)
X_out = transform_nan_encoding(X_out, num_feat, nan_encoding)

X_train = X_train[all_feats]
X_test = X_test[all_feats]
X_out = X_out[all_feats]

In [16]:
X_train.head()

,days_from_first_log_in,days_from_last_log_in,cnt_log_in_last_30_days,cnt_log_in_last_60_days,cnt_log_in_last_120_days,cnt_log_in_last_360_days,cnt_log_in_ever,cnt_ip_adress_in_30_days,cnt_ip_adress_in_60_days,cnt_ip_adress_in_120_days,...,requested_amount,industry_current,io_device_type,io_device_browser_type,settlement_type,reg_date,employment_y,ip,qty_appl,region
49707,549,0,8,8,13,41,78,7,7,11,...,1019890.0,0.520925,0.444275,0.444275,0.477186,0.406299,0.424822,1.0,2,0.447112
87839,1737,0,12,26,45,170,298,4,11,20,...,2600000.0,0.430473,0.444275,0.444275,0.398185,0.406299,0.582514,1.0,5,0.403800
78968,1171,0,22,53,121,338,687,10,21,43,...,750000.0,0.520925,0.444275,0.444275,0.398185,0.406299,0.043636,1.0,10,0.358318
65664,91,0,1,1,26,26,26,1,1,11,...,3500000.0,0.430473,0.444275,0.444275,0.398185,0.491365,0.582514,1.0,1,0.451613
17970,499,0,1,2,5,43,79,1,2,3,...,250000.0,0.520925,0.444275,0.444275,0.156222,0.406299,0.065574,1.0,3,0.448854


In [17]:
inf_encoding = fit_inf_encoding(X_train, num_feat)

In [18]:
X_train = transform_inf_encoding(X_train, num_feat, inf_encoding)
X_test = transform_inf_encoding(X_test, num_feat, inf_encoding)
X_out = transform_inf_encoding(X_out, num_feat, inf_encoding)

In [19]:
#Запись в файл
with open('result/preprocessing_rules.pkl', 'wb') as file:
    pickle.dump([nan_encoding, cat_encoding, inf_encoding], file)

In [50]:
#Чтение из файла
# with open('result/preprocessing_rules.pkl', 'rb') as file:
#     nan_encoding, cat_encoding, inf_encoding = pickle.load(file)

In [51]:
# Результирующий энкодер категориальных переменных - словарь словаей, со значениями для каждой категории каждого признака
nan_encoding, cat_encoding, inf_encoding

({'days_from_first_log_in': 955.0,
  'days_from_last_log_in': 0.0,
  'cnt_log_in_last_30_days': 6.0,
  'cnt_log_in_last_60_days': 10.0,
  'cnt_log_in_last_120_days': 19.0,
  'cnt_log_in_last_360_days': 51.0,
  'cnt_log_in_ever': 102.0,
  'cnt_ip_adress_in_30_days': 3.0,
  'cnt_ip_adress_in_60_days': 4.0,
  'cnt_ip_adress_in_120_days': 8.0,
  'cnt_ip_adress_in_360_days': 20.0,
  'cnt_ip_adress_ever': 42.0,
  'work_amount_x': 300000.0,
  'education': 3.0,
  'dependants_count': 1.0,
  'bor_age': 35.0,
  'sex_age': 60.0,
  'avg_longtitude_of_call': 0.0,
  'max_weights_of_call': 1.0,
  'min_weights_of_call': 0.0,
  'avg_weights_of_call': 0.8333333333333334,
  'cnt_of_error_call': 0.0,
  'cnt_of_shutdown_call': 0.0,
  'cnt_of_cancel_call': 0.0,
  'cnt_of_timeout_call': 0.0,
  'cnt_of_failure_call': 0.0,
  'cnt_of_failed_call': 0.0,
  'cnt_of_missed_call': 0.0,
  'cnt_of_success_call': 2.0,
  'cnt_of_abandoned_call': 0.0,
  'day_since_first_day_of_call': 597.0,
  'day_since_last_day_of_call':

### Отбор признаков

In [21]:
# Фильтруем признаки по следующему принципу:
# для каждого признака делаем однофакторный расчёт gini на train выборке для каждого месяца
# в разбивке по DATE (обязательно должен присутствовать в df_train).
# После этого отфильтровываем признаки, для которых gini меньше чем gini_min
# для количества интервалов большего num_bad_intervals.
# Т.е. например, если gini_min=0.05 и num_bad_intervals=2, то если gini переменной больше 0.05 для всех месяцев, кроме одного - это ок. 
gini_min = 0.01 # 0.05 - по дефолту норм, можно взять в диапозоне 0.02 - 0.08 в зависимости от качества признаков.
num_bad_intervals = 0  # 2 месяца/недели по дефолту норм, можно взять 10-20% от общего количества месяцев в сэмпле.
date_name = 'date_requested'  # Дата, по которой делить на месяцы.
intervals = 'month'  # Интервалы времени для разбивки, month или week

gini_feats, df_gini_months = gini_month_selection(X_train, df_train, gini_min=gini_min,
                                                  num_bad_intervals=num_bad_intervals, date_name=date_name,
                                                  intervals=intervals)

In [22]:
print(len(list(gini_feats)))
df_gini_months[df_gini_months['vars'].isin(gini_feats)].head()

82


,vars,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11
5,cnt_log_in_last_360_days,0.141,0.030,0.046,0.044,0.112,0.085,0.083,0.043
6,cnt_log_in_ever,0.300,0.184,0.191,0.198,0.208,0.228,0.114,0.079
9,cnt_ip_adress_in_120_days,0.230,0.096,0.057,0.016,0.089,0.036,0.103,0.069
10,cnt_ip_adress_in_360_days,0.173,0.055,0.096,0.075,0.087,0.101,0.077,0.040
11,cnt_ip_adress_ever,0.318,0.205,0.237,0.230,0.181,0.245,0.118,0.068


In [23]:
# Отбираем признаки по их значимости во время построения Деревьев решений.
top_n = 55  # Количество лучших признаков, которые оставить. Можно брать в диапозоне 45-80.
rf_feats, rf_imp = rf_feature_selection(X_train[gini_feats], y_train, top_n=top_n)

In [24]:
#Список признаков, который пойдёт на построения рулов (необязательно использовать только эти признаки)
rf_feats[:10]

['product_group',
 'Decl_last_days',
 'employment_y',
 'Decl_cnt_in_180_days',
 'comp_cnt_in_360_days',
 'Decl_cnt_in_90_days',
 'days_from_last_c_d_repaid',
 'comp_cnt_in_720_days',
 'employment_x',
 'reapeted_app_6m']

### Подбор рулов и выделение сегментов

#### На примере поиска рулов, для выделения сегмента с пониженным BR

In [25]:
y_train.mean()

0.4442506178223579

In [26]:
# Выполняем поиск различных рулов в указанном интервале
rules_trees_dwn = rules_search_interv(X_train, y_train, rf_feats, BR_interval=[0, 0.02]) #получившиеся рулы при отборе веток и дальнейшем их объединениии

In [27]:
rules_trees_dwn

,rule,BR,Segm_share
1897,(product_group <= 0.364)&(employment_y <= 0.50...,0.018,0.168
1022,(product_group <= 0.364)&(employment_y <= 0.50...,0.018,0.168
432,(product_group <= 0.364)&(employment_y <= 0.50...,0.018,0.168
1481,(product_group <= 0.364)&(employment_y <= 0.50...,0.020,0.162
428,(product_group <= 0.364)&(employment_y <= 0.50...,0.020,0.162
...,...,...,...
417,(days_from_last_c_d_repaid <= 0.5)&(cnt_in_com...,0.000,0.003
1178,(product_group > 0.364)&(stat_of_pred_cred <= ...,0.000,0.003
123,(product_group > 0.364)&(cnt_in_comp_credits <...,0.000,0.003
409,(comp_cnt_in_180_days > 0.5)&(credit_purpose <...,0.000,0.003


In [28]:
# Убираем повторяющиеся рулы(такое может быть)
rules_trees_dwn = rules_trees_dwn.drop_duplicates().sort_values('BR', ascending=True) #получившиеся рулы на отдельных объединённых ветках

# Фильтрую получившиеся рулы как мне нужно
rules_trees_dwn[(rules_trees_dwn['BR']<0.03)&(rules_trees_dwn['Segm_share']>0.1)]

,rule,BR,Segm_share
1142,(product_group <= 0.364)&(employment_y <= 0.50...,0.007,0.118
17,(product_group <= 0.364)&(employment_y <= 0.50...,0.008,0.107
751,(product_group <= 0.364)&(employment_y <= 0.50...,0.008,0.122
1560,(product_group <= 0.364)&(employment_y <= 0.50...,0.008,0.121
1008,(product_group <= 0.364)&(employment_y <= 0.50...,0.009,0.128
803,(product_group <= 0.364)&(employment_y <= 0.50...,0.010,0.137
1214,(Decl_cnt_in_90_days > 0.5)&(employment_y <= 0...,0.011,0.102
171,(product_group <= 0.364)&(employment_y <= 0.50...,0.011,0.141
1039,(Decl_last_days <= 91.5)&(employment_y <= 0.50...,0.011,0.103
1912,(reapeted_app_3w > 0.5)&(employment_y <= 0.504...,0.012,0.104


In [29]:
# Пересохранять не обязательно, просто для удобства работы дальше
rules_trees_dwn = rules_trees_dwn[(rules_trees_dwn['BR']<0.03)&(rules_trees_dwn['Segm_share']>0.1)]

In [31]:
# Выбираю как список с получившимися рулами
filters_list = list(rules_trees_dwn.rule.values)

In [32]:
filters_list

['(product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app_3w > 0.5)',
 '(product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_cnt_in_360_days > 2.5)',
 '(product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_last_days <= 91.5)',
 '(product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_cnt_in_90_days > 0.5)',
 '(product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app > 1.5)',
 '(product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app_6m > 0.5)',
 '(Decl_cnt_in_90_days > 0.5)&(employment_y <= 0.504)&(cnt_of_outbound_calls_in_30_days <= 1.5)&(employment_y <= 0.169)',
 '(product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_cnt_in_180_days > 0.5)',
 '(Decl_last_days <= 91.5)&(employment_y <= 0.504)&(cnt_of_outbound_calls_in_30_days <= 1.5)&(employment_y <= 0.169)',
 '(reapeted_app_3w > 0.5)&(employment_y 

In [33]:
full = check_rules(filters_list, X_train, y_train, X_test, y_test, X_out=X_out, y_out=y_out)
full

c:\Users\m.skrebtsov\Desktop\Projects\Rules_Framework\rules_searcher.py:286: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test['target'] = y_test
c:\Users\m.skrebtsov\Desktop\Projects\Rules_Framework\rules_searcher.py:292: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_out['target'] = y_out


,rule,br_train,segm_train,br_dwn_train,br_test,segm_test,br_dwn_test,br_out,segm_out,br_dwn_out
0,(product_group <= 0.364)&(employment_y <= 0.50...,0.007,0.118,0.017,0.012,0.122,0.027,0.015,0.080,0.033
1,(product_group <= 0.364)&(employment_y <= 0.50...,0.008,0.107,0.018,0.011,0.110,0.024,0.006,0.061,0.012
2,(product_group <= 0.364)&(employment_y <= 0.50...,0.008,0.122,0.019,0.012,0.125,0.026,0.015,0.082,0.032
3,(product_group <= 0.364)&(employment_y <= 0.50...,0.008,0.121,0.019,0.012,0.124,0.026,0.015,0.082,0.032
4,(product_group <= 0.364)&(employment_y <= 0.50...,0.009,0.128,0.019,0.007,0.132,0.015,0.011,0.076,0.025
5,(product_group <= 0.364)&(employment_y <= 0.50...,0.010,0.137,0.023,0.011,0.138,0.024,0.016,0.089,0.034
6,(Decl_cnt_in_90_days > 0.5)&(employment_y <= 0...,0.011,0.102,0.026,0.021,0.097,0.047,0.007,0.053,0.014
7,(product_group <= 0.364)&(employment_y <= 0.50...,0.011,0.141,0.026,0.010,0.141,0.023,0.015,0.092,0.033
8,(Decl_last_days <= 91.5)&(employment_y <= 0.50...,0.011,0.103,0.025,0.021,0.098,0.047,0.007,0.054,0.014
9,(reapeted_app_3w > 0.5)&(employment_y <= 0.504...,0.012,0.104,0.027,0.020,0.100,0.046,0.010,0.055,0.021


In [34]:
#Фильтруем итоговый список базовых рулов, их них уже можно выбрать какойто итоговый
full[(full['br_test'] < 0.02)&(full['br_out'] < 0.02)&(full['segm_out'] >= 0.0)]

,rule,br_train,segm_train,br_dwn_train,br_test,segm_test,br_dwn_test,br_out,segm_out,br_dwn_out
0,(product_group <= 0.364)&(employment_y <= 0.50...,0.007,0.118,0.017,0.012,0.122,0.027,0.015,0.080,0.033
1,(product_group <= 0.364)&(employment_y <= 0.50...,0.008,0.107,0.018,0.011,0.110,0.024,0.006,0.061,0.012
2,(product_group <= 0.364)&(employment_y <= 0.50...,0.008,0.122,0.019,0.012,0.125,0.026,0.015,0.082,0.032
3,(product_group <= 0.364)&(employment_y <= 0.50...,0.008,0.121,0.019,0.012,0.124,0.026,0.015,0.082,0.032
4,(product_group <= 0.364)&(employment_y <= 0.50...,0.009,0.128,0.019,0.007,0.132,0.015,0.011,0.076,0.025
5,(product_group <= 0.364)&(employment_y <= 0.50...,0.010,0.137,0.023,0.011,0.138,0.024,0.016,0.089,0.034
7,(product_group <= 0.364)&(employment_y <= 0.50...,0.011,0.141,0.026,0.010,0.141,0.023,0.015,0.092,0.033
10,(product_group <= 0.364)&(employment_y <= 0.50...,0.012,0.145,0.027,0.012,0.147,0.027,0.019,0.094,0.040
14,(product_group <= 0.364)&(employment_y <= 0.50...,0.015,0.114,0.035,0.018,0.111,0.041,0.019,0.072,0.042
16,(Decl_cnt_in_360_days > 1.5)&(employment_y <= ...,0.016,0.109,0.036,0.019,0.104,0.044,0.010,0.054,0.021


In [35]:


# Если есть необходимость исплользовать сложные рулы - состоящие из нескольких базовых 
# (такое больше нужно, когда сегменты плохо выделяются, и допустим доля отсекаемой выборки - составляет около 1% от исходной,
#  в этом случае логично попробовать пообъединять рулы, дабы повысить объём сегмента)




#### Жадный поиск

In [36]:
# Прогоняю через жадный поиск комбинаций рулов (1 вариант)
# filters_list - список используемых фильтров
# BR_interval - алгоритм будет пытаться подбирать такие рулы, чтобы BR результирующей выборки попадал в указанный интервал
# max_res_cnt_filters - максимальное количество фильтров, которое можно добавить в процессе жадного алгоритма
comb_greed = Greed_search_learn_interv(X_train, y_train, filters_list, BR_interval=[0, 0.02], max_res_cnt_filters=5)

In [37]:
res_greed = all_sample_data(X_train, y_train, X_test, y_test, filters_list, comb_greed, X_out=X_out, 
                    y_out=y_out
                    ) #значения в процентах
res_greed.head()

,filters,filt_cnt,BR_dwn_train,prt_train,BR_curr_train,BR_dwn_test,prt_test,BR_curr_test,BR_dwn_out,prt_out,BR_curr_out
0,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.341,11.833,0.737,97.313,12.18,1.193,96.699,8.006,1.525
1,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.169,10.721,0.814,97.63,11.047,1.053,98.756,6.07,0.575
2,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.121,12.189,0.835,97.382,12.5,1.163,96.784,8.216,1.486
3,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.099,12.051,0.844,97.351,12.355,1.176,96.777,8.198,1.489
4,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.085,12.814,0.851,98.512,13.198,0.661,97.529,7.64,1.142


In [38]:
# Тут просто фиксируем выбранный рул для нужного сегмента
res_greed[(res_greed['BR_curr_train']<1)&(res_greed['BR_curr_test']<2)&(res_greed['BR_curr_out']<10)]

,filters,filt_cnt,BR_dwn_train,prt_train,BR_curr_train,BR_dwn_test,prt_test,BR_curr_test,BR_dwn_out,prt_out,BR_curr_out
0,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.341,11.833,0.737,97.313,12.18,1.193,96.699,8.006,1.525
1,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.169,10.721,0.814,97.63,11.047,1.053,98.756,6.07,0.575
2,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.121,12.189,0.835,97.382,12.5,1.163,96.784,8.216,1.486
3,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.099,12.051,0.844,97.351,12.355,1.176,96.777,8.198,1.489
4,((product_group <= 0.364)&(employment_y <= 0.5...,1,98.085,12.814,0.851,98.512,13.198,0.661,97.529,7.64,1.142


#### Рандомный поиск

In [39]:
#Уменьшил список рулов, чтобы быстрее считался рандомный поиск (не делать при реальной работе)
filters_list = filters_list[:10]

In [40]:
# Прогоняю через рандомный поиск комбинаций рулов  (2 вариант)
comb_rand = random_step_by_step_learn_interv(X_train, y_train, filters_list, BR_interval=[0, 0.02])

In [41]:
res_rand = all_sample_data(X_train, y_train, X_test, y_test, filters_list, comb_rand,
                           X_out=X_out, y_out=y_out) #значения в процентах
res_rand

,filters,filt_cnt,BR_dwn_train,prt_train,BR_curr_train,BR_dwn_test,prt_test,BR_curr_test,BR_dwn_out,prt_out,BR_curr_out
0,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.903,13.265,0.932,96.567,13.343,1.525,96.982,8.756,1.394
1,((product_group <= 0.364)&(employment_y <= 0.5...,3,97.525,15.867,1.099,97.092,15.756,1.292,96.999,10.065,1.386
2,((product_group <= 0.364)&(employment_y <= 0.5...,4,97.004,16.383,1.331,97.186,16.279,1.25,97.119,10.483,1.331
3,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.744,13.054,1.002,96.552,13.285,1.532,96.509,8.652,1.613
4,((product_group <= 0.364)&(employment_y <= 0.5...,3,97.448,14.748,1.134,96.898,14.767,1.378,96.446,9.559,1.642
...,...,...,...,...,...,...,...,...,...,...,...
161,((product_group <= 0.364)&(employment_y <= 0.5...,5,96.726,16.492,1.454,97.225,16.512,1.232,96.786,10.57,1.485
162,((product_group <= 0.364)&(employment_y <= 0.5...,3,97.668,13.33,1.036,96.574,13.372,1.522,96.572,8.809,1.584
163,((product_group <= 0.364)&(employment_y <= 0.5...,4,97.181,15.089,1.252,96.928,14.913,1.365,96.497,9.698,1.619
164,((product_group <= 0.364)&(employment_y <= 0.5...,5,96.852,15.591,1.399,97.038,15.465,1.316,96.624,10.065,1.56


Рандомный поиск работает немного дольше, но различных комбинаций выдает намного больше, из-за чего есть что выбрать

In [42]:
res_rand[(res_rand['BR_curr_train']<1)&(res_rand['BR_curr_test']<2)&(res_rand['BR_curr_out']<10)]

,filters,filt_cnt,BR_dwn_train,prt_train,BR_curr_train,BR_dwn_test,prt_test,BR_curr_test,BR_dwn_out,prt_out,BR_curr_out
0,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.903,13.265,0.932,96.567,13.343,1.525,96.982,8.756,1.394
7,((product_group <= 0.364)&(employment_y <= 0.5...,2,98.122,12.197,0.834,97.382,12.5,1.163,96.784,8.216,1.486
8,((product_group <= 0.364)&(employment_y <= 0.5...,3,97.803,13.403,0.976,96.625,13.576,1.499,97.018,8.861,1.378
14,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.883,13.141,0.94,96.544,13.256,1.535,96.964,8.704,1.403
30,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.944,13.527,0.913,97.605,13.663,1.064,96.612,8.913,1.566
39,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.857,13.745,0.952,97.635,13.837,1.05,96.658,9.035,1.544
43,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.776,13.243,0.988,98.084,13.663,0.851,97.678,8.128,1.073
49,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.802,13.396,0.977,96.625,13.576,1.499,97.018,8.861,1.378
55,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.798,13.374,0.978,96.625,13.576,1.499,97.0,8.809,1.386
65,((product_group <= 0.364)&(employment_y <= 0.5...,2,97.848,13.687,0.956,97.62,13.75,1.057,96.658,9.035,1.544


In [43]:
rule_dwn = res_rand[(res_rand['BR_curr_train']<1)&(res_rand['BR_curr_test']<2)&(res_rand['BR_curr_out']<10)].filters[55]
rule_dwn

'((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_last_days <= 91.5))|((Decl_cnt_in_90_days > 0.5)&(employment_y <= 0.504)&(cnt_of_outbound_calls_in_30_days <= 1.5)&(employment_y <= 0.169))'

In [44]:
not_rule_dwn = '~(' + rule_dwn + ')'

X_train['target'] = y_train
print('train: BR в оставшейся выборке' , round(X_train.query(not_rule_dwn).target.mean(), 5), 'segm', round(X_train.query(not_rule_dwn).shape[0] /X_train.shape[0], 5))
print('train: BR в выделенной выборке' , round(X_train.query(rule_dwn).target.mean(), 5), 'segm', round(X_train.query(rule_dwn).shape[0] /X_train.shape[0], 5))
print('train: Кол-во гудов' , X_train.query(rule_dwn).shape[0] - X_train.query(rule_dwn).target.sum(), 'Кол-во бэдов', X_train.query(rule_dwn).target.sum())
print('train at all', round(y_train.mean(), 5))
print()

X_test['target'] = y_test
print('test: BR в оставшейся выборке' , round(X_test.query(not_rule_dwn).target.mean(), 5), 'segm', round(X_test.query(not_rule_dwn).shape[0] /X_test.shape[0], 5))
print('test: BR в выделенной выборке' , round(X_test.query(rule_dwn).target.mean(), 5), 'segm', round(X_test.query(rule_dwn).shape[0] /X_test.shape[0], 5))
print('test: Кол-во гудов' , X_test.query(rule_dwn).shape[0] - X_test.query(rule_dwn).target.sum(), 'Кол-во бэдов', X_test.query(rule_dwn).target.sum())
print('test at all', round(y_test.mean(), 5))
print()

X_out['target'] = y_out
print('out: BR в оставшейся выборке' , round(X_out.query(not_rule_dwn).target.mean(), 5), 'segm', round(X_out.query(not_rule_dwn).shape[0] /X_out.shape[0], 5))
print('out: BR в выделенной выборке' , round(X_out.query(rule_dwn).target.mean(), 5), 'segm', round(X_out.query(rule_dwn).shape[0] /X_out.shape[0], 5))
print('out: Кол-во гудов' , X_out.query(rule_dwn).shape[0] - X_out.query(rule_dwn).target.sum(), 'Кол-во бэдов', X_out.query(rule_dwn).target.sum())
print('out at all', round(y_out.mean(), 5))

X_alle = pd.concat([X_train, X_test, X_out])
print()

print('Доля гудов попавших в выделенную выборку:', round((X_alle.query(rule_dwn).shape[0] - X_alle.query(rule_dwn).target.sum())/(X_alle.shape[0] - X_alle.target.sum()), 5))
print('Доля бэдов попавших в выделенную выборку:', round((X_alle.query(rule_dwn).target.sum())/( X_alle.target.sum()), 5))
print('Кол-во гудов попавших в выделенную выборку:', X_alle.query(rule_dwn).shape[0] - X_alle.query(rule_dwn).target.sum())
print('Кол-во бэдов попавших в выделенную выборку:', X_alle.query(rule_dwn).target.sum())
print()
print('BR в исходной выборке', round(X_alle.target.mean(), 5))
print('BR в выделенной выборке(плохой):', round(X_alle.query(rule_dwn).target.mean(), 5))
print('BR в оставшейся выборке:', round(X_alle.query(not_rule_dwn).target.mean(), 5))
print('AR в исходном случае: ', round(X_alle.target.sum() / X_alle.shape[0], 3))
print('AR в новом случае: ', round((X_alle.target.sum()-X_alle.query(rule_dwn).target.sum()) / X_alle.shape[0], 3))

X_train.drop(columns=['target'], axis=1, inplace=True)
X_test.drop(columns=['target'], axis=1, inplace=True)
X_out.drop(columns=['target'], axis=1, inplace=True)

train: BR в оставшейся выборке 0.51133 segm 0.86626
train: BR в выделенной выборке 0.00978 segm 0.13374
train: Кол-во гудов 1822 Кол-во бэдов 18
train at all 0.44425

test: BR в оставшейся выборке 0.5116 segm 0.86424
test: BR в выделенной выборке 0.01499 segm 0.13576
test: Кол-во гудов 460 Кол-во бэдов 7
test at all 0.44419

out: BR в оставшейся выборке 0.50536 segm 0.91191
out: BR в выделенной выборке 0.01386 segm 0.08809
out: Кол-во гудов 498 Кол-во бэдов 7
out at all 0.46206

Доля гудов попавших в выделенную выборку: 0.2199
Доля бэдов попавших в выделенную выборку: 0.00311
Кол-во гудов попавших в выделенную выборку: 2780
Кол-во бэдов попавших в выделенную выборку: 32

BR в исходной выборке 0.44869
BR в выделенной выборке(плохой): 0.01138
BR в оставшейся выборке: 0.50982
AR в исходном случае:  0.449
AR в новом случае:  0.447


In [45]:
# Аналогичным образом можно набрать рулов для других сегментов с другим уровнем риска

#### Вывод результатов

In [46]:
# Допустим получились следующие рулы, просто для примера

f_dwn     =    '((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app_3w > 0.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app > 1.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_cnt_in_180_days > 0.5))|((Decl_last_days <= 91.5)&(employment_y <= 0.504)&(cnt_of_outbound_calls_in_30_days <= 1.5)&(employment_y <= 0.169))'
f_mid_dwn   =    '((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app_3w > 0.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_last_days <= 91.5))'
f_mid_up    =  '((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app_3w > 0.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_last_days <= 91.5))|((Decl_last_days <= 91.5)&(employment_y <= 0.504)&(cnt_of_outbound_calls_in_30_days <= 1.5)&(employment_y <= 0.169))'
f_up     =     '((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app_3w > 0.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_last_days <= 91.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_cnt_in_180_days > 0.5))|((Decl_last_days <= 91.5)&(employment_y <= 0.504)&(cnt_of_outbound_calls_in_30_days <= 1.5)&(employment_y <= 0.169))'
      

In [47]:
# Составляю такой список, с рулами, чтобы BR в них тоже, желательно, возрастал - это ни на что не влияет,
# просто так вывод результатов будет более красивый
f_all = [f_dwn, f_mid_dwn, f_mid_up, f_up]

In [48]:
rules_result(f_all, X_train, X_test, X_out, y_train, y_test, y_out)

Без фильтров: BR = 0.449

1 rule :   ((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app_3w > 0.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app > 1.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_cnt_in_180_days > 0.5))|((Decl_last_days <= 91.5)&(employment_y <= 0.504)&(cnt_of_outbound_calls_in_30_days <= 1.5)&(employment_y <= 0.169))
2 rule :   ((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app_3w > 0.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_last_days <= 91.5))
3 rule :   ((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(reapeted_app_3w > 0.5))|((product_group <= 0.364)&(employment_y <= 0.504)&(product_group > 0.227)&(Decl_last_days <= 91.5))|((Decl_last_days <= 91.5)&(employment_y <= 0.504)&(cnt_of_outbound_calls_in_30_days <= 1.5)&(employment_y <= 0.169))
4 rule :  